# 军事通信效能评估 - 响应能力指标分析

## 步骤一 基于灰色关联分析的指标权重确定与综合评价

**数据库**: `military_communication_effectiveness`  
**分析指标**:
- 平均呼叫建立时长 (`avg_call_setup_duration_ms`)
- 平均传输时延 (`avg_transmission_delay_ms`)

**分析方法**:
1. Z-score异常值检测（95%置信区间）
2. Sigmoid归一化
3. 灰色关联分析（GRA）确定指标权重
4. 综合评价与排名

In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
军事通信效能评估 - 响应能力指标分析
基于灰色关联分析的指标权重确定与综合评价
"""

import mysql.connector
import pandas as pd
import numpy as np
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✓ 库导入成功")
print(f"分析时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✓ 库导入成功
分析时间: 2026-02-02 09:44:50


In [9]:
# 数据库配置
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root',
    'database': 'military_communication_effectiveness',
    'charset': 'utf8mb4'
}

def get_db_connection():
    """获取数据库连接"""
    return mysql.connector.connect(**DB_CONFIG)

print("✓ 数据库配置完成")

✓ 数据库配置完成


## 阶段1: 数据准备

从MySQL数据库读取通信数据，按实验ID分组计算平均值

## 数据标记说明

用于标记数据，方便后面识别字段：

- **响应能力 (Responsiveness)** → **RS**
- **处理能力 (Processing)** → **PO**
- 有效性 (Effectiveness) → EF
- 可靠性 (Reliability) → RL
- 抗干扰性 (Anti-jamming) → AJ
- 人为操作 (Human Operation) → HO
- 组网能力 (Networking Capability) → NC
- 机动性 (Mobility) → MB
- 安全性 (Security) → SC

In [10]:
def load_data_from_mysql():
    """
    从MySQL读取数据并按test_id分组计算平均值
    包含响应能力(RS)和处理能力(PO)指标
    """
    conn = get_db_connection()
    
    # SQL查询：按test_id分组计算平均值
    query = """
    SELECT 
        test_id,
        
        -- 响应能力 (Responsiveness - RS)
        AVG(call_setup_duration_ms) as avg_call_setup_duration_ms_RS,
        AVG(transmission_delay_ms) as avg_transmission_delay_ms_RS,
        
        -- 处理能力 (Processing - PO)
        AVG(instant_throughput) as effective_throughput_PO,
        AVG(instant_throughput / channel_bandwidth) as spectral_efficiency_PO,
        
        -- 辅助统计
        COUNT(*) as communication_count,
        AVG(channel_bandwidth) as avg_bandwidth
        
    FROM during_battle_communications
    GROUP BY test_id
    ORDER BY test_id
    """
    
    # 读取数据
    df = pd.read_sql(query, conn)
    conn.close()
    
    print(f"✓ 成功读取 {len(df)} 个实验的数据")
    print(f"\n响应能力(RS)指标:")
    print(f"  - avg_call_setup_duration_ms_RS (平均呼叫建立时长)")
    print(f"  - avg_transmission_delay_ms_RS (平均传输时延)")
    print(f"\n处理能力(PO)指标:")
    print(f"  - effective_throughput_PO (有效吞吐量)")
    print(f"  - spectral_efficiency_PO (频谱效率)")
    
    return df

# 执行数据读取
df_raw = load_data_from_mysql()
print("\n数据预览:")
print(df_raw[['test_id', 'avg_call_setup_duration_ms_RS', 'avg_transmission_delay_ms_RS', 
              'effective_throughput_PO', 'spectral_efficiency_PO']].head(10))

✓ 成功读取 10 个实验的数据

响应能力(RS)指标:
  - avg_call_setup_duration_ms_RS (平均呼叫建立时长)
  - avg_transmission_delay_ms_RS (平均传输时延)

处理能力(PO)指标:
  - effective_throughput_PO (有效吞吐量)
  - spectral_efficiency_PO (频谱效率)

数据预览:
         test_id  avg_call_setup_duration_ms_RS  avg_transmission_delay_ms_RS  \
0  TEST-2026-001                    1468.531357                    316.940643   
1  TEST-2026-002                    1475.829250                    277.450250   
2  TEST-2026-003                    1572.638964                    289.538000   
3  TEST-2026-004                    1588.183267                    318.911467   
4  TEST-2026-005                    1402.650857                    310.988214   
5  TEST-2026-006                    1609.146000                    284.714036   
6  TEST-2026-007                    1416.720185                    336.915333   
7  TEST-2026-008                    1516.849577                    308.388038   
8  TEST-2026-009                    1511.160821                 

## 阶段2: Z-score分析与最优值选择

计算Z-score，标记95%置信区间，从正常范围内选择最优值

- **响应能力**: 越小越好，选择MIN
- **处理能力**: 越大越好，选择MAX

In [18]:
def calculate_zscore_and_optimal(df, metrics_config):
    """
    计算Z-score并选择最优值
    
    参数:
        df: 数据框
        metrics_config: 指标配置字典
            {
                'metric_name': {
                    'direction': 'min' 或 'max',  # 优化方向
                    'label': '指标显示名称'
                }
            }
    
    返回:
        df_result: 包含Z-score的数据框
        optimal_values: 最优值字典
        statistics: 统计信息字典
    """
    df_result = df.copy()
    optimal_values = {}
    statistics = {}
    
    for metric, config in metrics_config.items():
        # 计算统计量
        mean_val = df[metric].mean()
        std_val = df[metric].std()
        
        # 计算Z-score
        df_result[f'{metric}_zscore'] = (df[metric] - mean_val) / std_val
        
        # 标记95%置信区间 (|Z-score| <= 1.96)
        df_result[f'{metric}_in_ci95'] = df_result[f'{metric}_zscore'].abs() <= 1.96
        
        # 从95%置信区间内选择最优值
        df_in_ci = df_result[df_result[f'{metric}_in_ci95']]
        
        if config['direction'] == 'min':
            # 越小越好（响应能力）
            optimal_value = df_in_ci[metric].min()
        else:
            # 越大越好（处理能力）
            optimal_value = df_in_ci[metric].max()
        
        optimal_test_id = df_in_ci[df_in_ci[metric] == optimal_value]['test_id'].values[0]
        
        # 保存结果
        optimal_values[metric] = {
            'value': optimal_value,
            'test_id': optimal_test_id,
            'direction': config['direction']
        }
        
        statistics[metric] = {
            'mean': mean_val,
            'std': std_val,
            'min': df[metric].min(),
            'max': df[metric].max(),
            'ci_95_lower': mean_val - 1.96 * std_val,
            'ci_95_upper': mean_val + 1.96 * std_val,
            'normal_count': df_result[f'{metric}_in_ci95'].sum(),
            'outlier_count': (~df_result[f'{metric}_in_ci95']).sum(),
            'direction': config['direction'],
            'label': config['label']
        }
        
        # 标记是否为最优实验
        df_result[f'{metric}_is_optimal'] = df_result['test_id'] == optimal_test_id
        
        # 计算与最优值的差距
        df_result[f'{metric}_diff_from_optimal'] = df_result[metric] - optimal_value
        if config['direction'] == 'min':
            # 越小越好：正差距表示比最优值大（不好）
            df_result[f'{metric}_diff_percentage'] = (df_result[f'{metric}_diff_from_optimal'] / optimal_value) * 100
        else:
            # 越大越好：负差距表示比最优值小（不好）
            df_result[f'{metric}_diff_percentage'] = (df_result[f'{metric}_diff_from_optimal'] / optimal_value) * 100
    
    return df_result, optimal_values, statistics

# 配置指标
metrics_config = {
    # 响应能力 (RS) - 越小越好
    'avg_call_setup_duration_ms_RS': {
        'direction': 'min',
        'label': '平均呼叫建立时长(RS)'
    },
    'avg_transmission_delay_ms_RS': {
        'direction': 'min',
        'label': '平均传输时延(RS)'
    },
    # 处理能力 (PO) - 越大越好
    'effective_throughput_PO': {
        'direction': 'max',
        'label': '有效吞吐量(PO)'
    },
    'spectral_efficiency_PO': {
        'direction': 'max',
        'label': '频谱效率(PO)'
    }
}

# 执行Z-score分析
df_zscore, optimal_values, statistics = calculate_zscore_and_optimal(df_raw, metrics_config)

print("=" * 80)
print("Z-score分析结果")
print("=" * 80)

print("【响应能力 (RS) - 越小越好】")
for metric in ['avg_call_setup_duration_ms_RS', 'avg_transmission_delay_ms_RS']:
    print(f"\n{statistics[metric]['label']}:")
    print(f"  均值 (μ): {statistics[metric]['mean']:.3f}")
    print(f"  标准差 (σ): {statistics[metric]['std']:.3f}")
    print(f"  95%置信区间: [{statistics[metric]['ci_95_lower']:.3f}, {statistics[metric]['ci_95_upper']:.3f}]")
    print(f"  最优值: {optimal_values[metric]['value']:.3f} (来自 {optimal_values[metric]['test_id']})")
    print(f"  正常实验数: {statistics[metric]['normal_count']}")
    print(f"  异常实验数: {statistics[metric]['outlier_count']}")

print("【处理能力 (PO) - 越大越好】")
for metric in ['effective_throughput_PO', 'spectral_efficiency_PO']:
    print(f"{statistics[metric]['label']}:")
    print(f"  均值 (μ): {statistics[metric]['mean']:.3f}")
    print(f"  标准差 (σ): {statistics[metric]['std']:.3f}")
    print(f"  95%置信区间: [{statistics[metric]['ci_95_lower']:.3f}, {statistics[metric]['ci_95_upper']:.3f}]")
    print(f"  最优值: {optimal_values[metric]['value']:.3f} (来自 {optimal_values[metric]['test_id']})")
    print(f"  正常实验数: {statistics[metric]['normal_count']}")
    print(f"  异常实验数: {statistics[metric]['outlier_count']}")

print("\n" + "="*80)
print("Z-score数据预览:")
print("="*80)
print(df_zscore[['test_id', 
                 'avg_call_setup_duration_ms_RS', 'avg_call_setup_duration_ms_RS_zscore', 'avg_call_setup_duration_ms_RS_in_ci95',
                 'effective_throughput_PO', 'effective_throughput_PO_zscore', 'effective_throughput_PO_in_ci95']].head(10))

Z-score分析结果
【响应能力 (RS) - 越小越好】

平均呼叫建立时长(RS):
  均值 (μ): 1523.022
  标准差 (σ): 85.930
  95%置信区间: [1354.598, 1691.446]
  最优值: 1402.651 (来自 TEST-2026-005)
  正常实验数: 10
  异常实验数: 0

平均传输时延(RS):
  均值 (μ): 306.963
  标准差 (σ): 22.910
  95%置信区间: [262.059, 351.868]
  最优值: 277.450 (来自 TEST-2026-002)
  正常实验数: 10
  异常实验数: 0
【处理能力 (PO) - 越大越好】
有效吞吐量(PO):
  均值 (μ): 29916.459
  标准差 (σ): 1430.952
  95%置信区间: [27111.792, 32721.125]
  最优值: 32187.679 (来自 TEST-2026-010)
  正常实验数: 10
  异常实验数: 0
频谱效率(PO):
  均值 (μ): 0.621
  标准差 (σ): 0.063
  95%置信区间: [0.497, 0.745]
  最优值: 0.712 (来自 TEST-2026-007)
  正常实验数: 10
  异常实验数: 0

Z-score数据预览:
         test_id  avg_call_setup_duration_ms_RS  \
0  TEST-2026-001                    1468.531357   
1  TEST-2026-002                    1475.829250   
2  TEST-2026-003                    1572.638964   
3  TEST-2026-004                    1588.183267   
4  TEST-2026-005                    1402.650857   
5  TEST-2026-006                    1609.146000   
6  TEST-2026-007                 

## 阶段3: 均值归一化（消除量纲影响）

### 公式

#### x' = x / μ

其中:
- x = 当前值
- μ = 均值（所有实验的平均值）
- x' = 归一化后的值

### 特点

✅ **线性变换**，保持原始数据的比例关系  
✅ **均值归一化后为 1.0**  
✅ **所有值都是正数**  
✅ **适合灰色关联分析**

### 为什么不用Sigmoid归一化？

❌ Sigmoid是**非线性变换**，会改变原始数据的比例关系  
❌ 在灰色关联分析中，需要保持与最优值的真实距离比例  
❌ Sigmoid会"压缩"异常值的影响，导致关联度计算失真

### 归一化效果

- **归一化后**: 所有指标都在 0.5~2.0 的相近数量级
- **量纲影响已消除**，可以进行灰色关联分析！


In [19]:
def mean_normalization(df, metrics_config, statistics):
    """
    均值归一化
    
    公式: x' = x / μ
    
    特点:
      - 线性变换，保持原始数据的比例关系
      - 均值归一化后为 1.0
      - 适合灰色关联分析
    """
    df_result = df.copy()
    normalization_params = {}
    
    for metric, config in metrics_config.items():
        mean_val = statistics[metric]['mean']
        
        # 均值归一化
        df_result[f'{metric}_normalized'] = df_result[metric] / mean_val
        
        normalization_params[metric] = {
            'mean': mean_val,
            'method': 'mean_normalization'
        }
    
    return df_result, normalization_params

# 执行均值归一化
df_normalized, normalization_params = mean_normalization(df_zscore, metrics_config, statistics)

print("\n" + "=" * 80)
print("均值归一化结果")
print("=" * 80)

print("\n归一化参数:")
for metric, params in normalization_params.items():
    print(f"\n{statistics[metric]['label']}:")
    print(f"  均值 (μ): {params['mean']:.3f}")
    print(f"  归一化方法: {params['method']}")
    print(f"  最优值归一化后: {optimal_values[metric]['value'] / params['mean']:.6f}")

print("\n" + "="*80)
print("归一化数据预览:")
print("="*80)
display_cols = ['test_id', 
                'avg_call_setup_duration_ms_RS', 'avg_call_setup_duration_ms_RS_normalized',
                'effective_throughput_PO', 'effective_throughput_PO_normalized']
print(df_normalized[display_cols].to_string(index=False))


均值归一化结果

归一化参数:

平均呼叫建立时长(RS):
  均值 (μ): 1523.022
  归一化方法: mean_normalization
  最优值归一化后: 0.920966

平均传输时延(RS):
  均值 (μ): 306.963
  归一化方法: mean_normalization
  最优值归一化后: 0.903855

有效吞吐量(PO):
  均值 (μ): 29916.459
  归一化方法: mean_normalization
  最优值归一化后: 1.075919

频谱效率(PO):
  均值 (μ): 0.621
  归一化方法: mean_normalization
  最优值归一化后: 1.147616

归一化数据预览:
      test_id  avg_call_setup_duration_ms_RS  avg_call_setup_duration_ms_RS_normalized  effective_throughput_PO  effective_throughput_PO_normalized
TEST-2026-001                    1468.531357                                  0.964222             28107.351071                            0.939528
TEST-2026-002                    1475.829250                                  0.969014             31592.459286                            1.056023
TEST-2026-003                    1572.638964                                  1.032578             29012.866071                            0.969796
TEST-2026-004                    1588.183267                     

## 阶段4: 灰色关联分析 - 第一阶段（确定指标权重）

通过计算每个指标与理想值的关联度来确定指标权重

In [20]:
def calculate_grey_relational_weights_normalized(df, dimension_metrics, optimal_values, 
                                                 normalization_params, dimension_name, rho=0.5):
    """
    灰色关联分析 - 使用归一化后的数据计算权重
    
    参数:
        df: 数据框
        dimension_metrics: 该维度的指标列表
        optimal_values: 最优值字典
        normalization_params: 归一化参数
        dimension_name: 维度名称
        rho: 分辨系数，默认0.5
    
    返回:
        weights: 指标权重字典
        grey_coefficients: 灰色关联系数矩阵
        relational_degrees: 平均关联度字典
    """
    n_experiments = len(df)
    grey_coefficients = {}
    relational_degrees = {}
    
    print(f"\n{'='*80}")
    print(f"灰色关联分析 - {dimension_name}")
    print(f"{'='*80}")
    
    for metric in dimension_metrics:
        # 归一化后的参考序列（最优值）
        optimal_normalized = optimal_values[metric]['value'] / normalization_params[metric]['mean']
        reference = np.full(n_experiments, optimal_normalized)
        
        # 归一化后的比较序列
        comparison = df[f'{metric}_normalized'].values
        
        # 计算差异序列
        delta = np.abs(reference - comparison)
        
        # 全局最小差和最大差
        delta_min = delta.min()
        delta_max = delta.max()
        
        # 计算灰色关联系数
        xi = (delta_min + rho * delta_max) / (delta + rho * delta_max)
        
        grey_coefficients[metric] = xi
        r = xi.mean()
        relational_degrees[metric] = r
        
        print(f"\n指标: {metric}")
        print(f"  原始最优值: {optimal_values[metric]['value']:.3f}")
        print(f"  归一化最优值: {optimal_normalized:.6f}")
        print(f"  Δmin: {delta_min:.6f}")
        print(f"  Δmax: {delta_max:.6f}")
        print(f"  平均关联度 (r): {r:.6f}")
    
    # 计算权重
    total_r = sum(relational_degrees.values())
    weights = {metric: relational_degrees[metric] / total_r for metric in dimension_metrics}
    
    print(f"\n{dimension_name} - 指标权重:")
    for metric, weight in weights.items():
        print(f"  {metric}: {weight:.6f} ({weight*100:.2f}%)")
    
    return weights, grey_coefficients, relational_degrees

# 响应能力(RS)维度
rs_metrics = ['avg_call_setup_duration_ms_RS', 'avg_transmission_delay_ms_RS']
rs_weights, rs_grey_coefficients, rs_relational_degrees = calculate_grey_relational_weights_normalized(
    df_normalized, rs_metrics, optimal_values, normalization_params, "响应能力(RS)维度"
)

# 处理能力(PO)维度
po_metrics = ['effective_throughput_PO', 'spectral_efficiency_PO']
po_weights, po_grey_coefficients, po_relational_degrees = calculate_grey_relational_weights_normalized(
    df_normalized, po_metrics, optimal_values, normalization_params, "处理能力(PO)维度"
)


灰色关联分析 - 响应能力(RS)维度

指标: avg_call_setup_duration_ms_RS
  原始最优值: 1402.651
  归一化最优值: 0.920966
  Δmin: 0.000000
  Δmax: 0.174560
  平均关联度 (r): 0.588754

指标: avg_transmission_delay_ms_RS
  原始最优值: 277.450
  归一化最优值: 0.903855
  Δmin: 0.000000
  Δmax: 0.213907
  平均关联度 (r): 0.600688

响应能力(RS)维度 - 指标权重:
  avg_call_setup_duration_ms_RS: 0.494983 (49.50%)
  avg_transmission_delay_ms_RS: 0.505017 (50.50%)

灰色关联分析 - 处理能力(PO)维度

指标: effective_throughput_PO
  原始最优值: 32187.679
  归一化最优值: 1.075919
  Δmin: 0.000000
  Δmax: 0.145221
  平均关联度 (r): 0.546486

指标: spectral_efficiency_PO
  原始最优值: 0.712
  归一化最优值: 1.147616
  Δmin: 0.000000
  Δmax: 0.343452
  平均关联度 (r): 0.589485

处理能力(PO)维度 - 指标权重:
  effective_throughput_PO: 0.481074 (48.11%)
  spectral_efficiency_PO: 0.518926 (51.89%)


## 阶段5: 灰色关联分析 - 第二阶段（综合评价）

使用确定的权重计算每个实验的加权关联度，进行综合评价

In [21]:
def calculate_comprehensive_scores(df, rs_metrics, po_metrics, rs_weights, po_weights, 
                                   rs_grey_coefficients, po_grey_coefficients):
    """
    计算综合评分
    
    参数:
        df: 数据框
        rs_metrics: 响应能力指标列表
        po_metrics: 处理能力指标列表
        rs_weights: 响应能力权重
        po_weights: 处理能力权重
        rs_grey_coefficients: 响应能力灰色关联系数
        po_grey_coefficients: 处理能力灰色关联系数
    
    返回:
        df_result: 包含评分的数据框
    """
    df_result = df.copy()
    
    # 为每个指标添加关联系数列
    for metric in rs_metrics:
        df_result[f'{metric}_grey_coefficient'] = rs_grey_coefficients[metric]
    
    for metric in po_metrics:
        df_result[f'{metric}_grey_coefficient'] = po_grey_coefficients[metric]
    
    # 计算响应能力得分
    rs_score = np.zeros(len(df))
    for metric in rs_metrics:
        rs_score += rs_weights[metric] * rs_grey_coefficients[metric]
    df_result['score_RS'] = rs_score
    
    # 计算处理能力得分
    po_score = np.zeros(len(df))
    for metric in po_metrics:
        po_score += po_weights[metric] * po_grey_coefficients[metric]
    df_result['score_PO'] = po_score
    
    # 计算综合得分（两个维度平均）
    df_result['score_total'] = (df_result['score_RS'] + df_result['score_PO']) / 2
    
    # 排名
    df_result['rank'] = df_result['score_total'].rank(ascending=False, method='min').astype(int)
    
    # 评价等级
    def get_grade(score):
        if score >= 0.9:
            return '优秀'
        elif score >= 0.8:
            return '良好'
        elif score >= 0.6:
            return '中等'
        else:
            return '较差'
    
    df_result['evaluation_grade'] = df_result['score_total'].apply(get_grade)
    
    # 按排名排序
    df_result = df_result.sort_values('rank')
    
    return df_result

# 执行综合评价
df_final = calculate_comprehensive_scores(
    df_normalized, rs_metrics, po_metrics, 
    rs_weights, po_weights, 
    rs_grey_coefficients, po_grey_coefficients
)

print("\n" + "=" * 80)
print("综合评价结果")
print("=" * 80)

print("\n实验排名:")
result_cols = ['rank', 'test_id', 'score_RS', 'score_PO', 'score_total', 'evaluation_grade']
print(df_final[result_cols].to_string(index=False))


综合评价结果

实验排名:
 rank       test_id  score_RS  score_PO  score_total evaluation_grade
    1 TEST-2026-002  0.824257  0.674587     0.749422               中等
    2 TEST-2026-010  0.600681  0.854284     0.727482               中等
    3 TEST-2026-007  0.627249  0.760941     0.694095               中等
    4 TEST-2026-006  0.607376  0.574498     0.590937               较差
    5 TEST-2026-005  0.744801  0.333333     0.539067               较差
    6 TEST-2026-009  0.440862  0.623300     0.532081               较差
    7 TEST-2026-004  0.429786  0.601727     0.515757               较差
    8 TEST-2026-001  0.560215  0.441014     0.500615               较差
    9 TEST-2026-003  0.586329  0.403308     0.494818               较差
   10 TEST-2026-008  0.526254  0.421003     0.473629               较差
